In [2]:
import tsplib95
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import csv
import time
import pandas as pd
from IPython.display import clear_output

In [3]:
def load_tsp_file_with_txt(file_path):
    problem = tsplib95.load(file_path)
    
    # get node coordinates
    coordinates = []
    for node in problem.get_nodes():
        coord = problem.node_coords[node]
        coordinates.append((coord[0], coord[1]))
    
    return np.array(coordinates)

# Load the data
cities = load_tsp_file_with_txt('a280.txt')

In [4]:
def euclidean_distance(city1, city2):
    return np.linalg.norm(city1 - city2)

def compute_distance_matrix(cities):
    """Computes distances between all cities 
    Returns: Distance matrix 
    """
    num_cities = len(cities)
    dist_matrix = np.zeros((num_cities, num_cities))
    for i in range(num_cities):
        for j in range(num_cities):
            if i != j:
                dist_matrix[i][j] = euclidean_distance(cities[i], cities[j])
    return dist_matrix

distance_matrix = compute_distance_matrix(cities)

def calculate_total_distance(tour, dist_matrix):
    """Calculate total distance of one tour (defined as array of cities)"""
    total_distance = 0
    num_cities = len(tour)
    for i in range(num_cities):
        total_distance += dist_matrix[tour[i]][tour[(i + 1) % num_cities]]
    return total_distance

def generate_initial_solution(num_cities):
    """Initialize a random tour"""
    tour = list(range(num_cities))
    random.shuffle(tour)
    return tour


initial_tour = generate_initial_solution(len(cities))

In [5]:
def cooling_schedule(cooling_type, k, N, initial_temperature, final_temperature):
    """
    Cooling schedule function for different cooling types, adjusted to reach final temperature in N steps.
    
    Parameters:
    - cooling_type (str): Type of cooling ('linear', 'geometric', 'exponential', 'logarithmic').
    - k (int): Current iteration (1-based).
    - N (int): Total number of timesteps for cooling.
    - initial_temperature (float): Starting temperature.
    - final_temperature (float): Target final temperature.
    
    Returns:
    - new_temperature (float): Updated temperature after cooling step.
    """
    # Linear cooling
    if cooling_type == "linear":
        step_size = (initial_temperature - final_temperature) / N
        new_temperature = initial_temperature - k * step_size
        return max(new_temperature, final_temperature)  # Ensure temperature does not go below final_temperature

    # Geometric cooling
    elif cooling_type == "geometric":
        cooling_rate = (final_temperature / initial_temperature) ** (1 / N)
        return initial_temperature * (cooling_rate ** k)

    # Exponential cooling
    elif cooling_type == "exponential":
        c = -N / math.log(final_temperature / initial_temperature)
        return initial_temperature * math.exp(-k / c)

    # Logarithmic cooling
    elif cooling_type == "logarithmic":
        beta = 1  # Scaling factor to reduce initial decay
        delta = 0  # Offset to smooth early transitions

        # Dynamically compute alpha to ensure T_final is reached after N steps
        alpha = (initial_temperature - final_temperature) / (final_temperature * math.log(beta * (N + 1) + delta))
    
        # Ensure no division by zero or logarithm domain errors
        if k == 0:
            return initial_temperature

        # Compute the new temperature
        new_temperature = initial_temperature / (1 + alpha * math.log(beta * (k + 1) + delta))

        return max(new_temperature, final_temperature)  # Ensure temperature doesn't drop below final_temperature
    else:
        raise ValueError("Invalid cooling type")

In [6]:
# Operators
def two_opt_swap(tour, n):
    """Perform a 2-opt swap by reversing the segment between i and k."""

    i, k = sorted(random.sample(range(n), 2))
    return tour[:i] + tour[i:k + 1][::-1] + tour[k + 1:]

def three_opt_swap(tour, n):
    """Perform a 3-opt swap."""

    i, j, k = sorted(random.sample(range(n), 3))
    new_tours = [
        tour[:i] + tour[i:j][::-1] + tour[j:k][::-1] + tour[k:],  # Reverse segments (i:j) and (j:k)
        tour[:i] + tour[j:k] + tour[i:j] + tour[k:],              # Swap segments (i:j) and (j:k)
        tour[:i] + tour[j:k][::-1] + tour[i:j] + tour[k:],        # Reverse segment (j:k), swap with (i:j)
    ]
    return random.choice(new_tours)

def move_city(tour, n):
    """Move a single city (node) to another location in the tour."""
    i, j = random.sample(range(n), 2)
    city = tour.pop(i)
    tour.insert(j, city)
    return tour

def inverse(tour, n):
    """Invert a random segment of the tour."""
    i, j = sorted(random.sample(range(n), 2))
    return tour[:i] + tour[i:j][::-1] + tour[j:]

def insert(tour, n):
    """Remove a city and insert it at a new position."""
    i, j = random.sample(range(n), 2)
    city = tour.pop(i)
    tour.insert(j, city)
    return tour

def swap(tour, n):
    """Swap two cities in the tour."""
    i, j = random.sample(range(n), 2)
    tour[i], tour[j] = tour[j], tour[i]
    return tour

def simulated_annealing(cities, initial_temperature, final_temperature, N, markov_length, cooling_type="geometric"):
    """Simulated annealing for solving TSP using various operators.
    
    Input: 
    cities (shape=(n,2) np.array) : coordinates of cities
    initial_temperature (float): Starting temperature
    cooling_rate: Cooling rate per step
    markov_length: Number of candidate solutions generated per temperature step
    cooling_type: Type of cooling function ('geometric', 'linear', 'exponential', 'logarithmic')
    
    Returns:
    best_solution: Best tour found
    distances: List of distances during optimization
    temperatures: List of temperatures during optimization
    iterations: Number of iterations
    """
    num_cities = len(cities)
    dist_matrix = compute_distance_matrix(cities)

    # Initial solution
    current_solution = generate_initial_solution(num_cities)
    current_distance = calculate_total_distance(current_solution, dist_matrix)
    best_solution = current_solution[:]
    best_distance = current_distance

    temperature = initial_temperature
    iteration = 0
    distances = []
    temperatures = []
    iterations = []

    while temperature > final_temperature:
        iteration += 1
        for _ in range(markov_length):
            # Randomly select operator based on weights
            # High weight for 2-opt and low weights for other operators
            # Swap is not recommended in assignment, so probably do not use
            operator = random.choices(
                ["2-opt", "3-opt", "inverse", "insert", "swap", "move"], 
                weights=[100, 0, 0, 0, 0, 0]
            )[0]
            
            if operator != "2-opt":
                print(operator)

            # Apply the operator to generate a candidate solution
            if operator == "2-opt":
                new_solution = two_opt_swap(current_solution, num_cities)
            elif operator == "3-opt":
                new_solution = three_opt_swap(current_solution, num_cities)
            elif operator == "inverse":
                new_solution = inverse(current_solution, num_cities)
            elif operator == "insert":
                new_solution = insert(current_solution, num_cities)
            elif operator == "swap":
                new_solution = swap(current_solution, num_cities)
            elif operator == "move":
                new_solution = move_city(current_solution, num_cities)

            new_distance = calculate_total_distance(new_solution, dist_matrix)
            delta = new_distance - current_distance

            # Accept the new solution with a probability
            if delta < 0 or random.random() < math.exp(-delta / temperature):
                current_solution = new_solution[:]
                current_distance = new_distance
    
                if current_distance < best_distance:
                    best_solution = current_solution[:]
                    best_distance = current_distance

        temperatures.append(temperature)
        distances.append(current_distance)
        iterations.append(iteration)

        # Cool down the temperature
        #print(f"iteration={iteration}, distance ={current_distance}")
        temperature = cooling_schedule(cooling_type, iteration, N, initial_temperature, final_temperature)

    return best_solution, distances, temperatures, iterations

In [53]:
def run_simulated_annealing_statistics(cities, initial_temperature, final_temperature, N, markov_length, cooling_type, N_runs):
    all_distances = []  # Store distances for all runs
    all_temperatures = []  # Store temperatures for all runs
    min_distances = []  # Store the final distances of each run

    for i in range(N_runs):
        best_solution, distances, temperatures, iterations = simulated_annealing(cities, initial_temperature, final_temperature, N, markov_length, cooling_type)
        min_distances.append(distances[-1])  # Save the final distance
        all_distances.append(distances)  # Save all distances for this run
        all_temperatures.append(temperatures)  # Save all temperatures for this run
        #print(f"Run {i + 1}/{N_runs}: Best Distance = {distances[-1]:.2f}")

    return min_distances, all_distances, all_temperatures

chain_min_distances = []
chain_full_distances = []
chain_full_temperatures = []

for chain_length in [10, 100, 250, 500, 750, 1000, 1500, 2000, 3500, 5000]:
    # Example usage:
    initial_temperature = 1000
    final_temperature = 0.01
    N = 1000  # Total number of timesteps

    cooling_rate = 0.995
    markov_length = chain_length
    cooling_type = "linear"
    N_runs = 10 # Number of runs

    min_distances, all_distances, all_temperatures = run_simulated_annealing_statistics(
        cities, initial_temperature, final_temperature, N, markov_length, cooling_type, N_runs
    )
    
    chain_full_distances.append(all_distances)
    chain_min_distances.append(min_distances)
    chain_full_temperatures.append(all_temperatures)
    print(chain_length, 'done')

10 done
100 done
250 done
500 done
750 done
1000 done
1500 done
2000 done
3500 done
5000 done


In [ ]:
#print(len(chain_full_distances))

# plot of minimum distances over chain lengths
# plot of distance curves for different chain length w/ confidence intervals (N = 10)
num_runs = 25
initial_temperature = 6000
final_temperature = 0.01
N = 10000

# Not sure which markov lengths we want to check?
markov_lengths = [10, 100, 250, 500, 750, 1000, 1500, 2000, 3500, 5000]

cooling_type = "geometric"

csv_filename = f'simulation_results/simulation_results_a280_{initial_temperature}_{cooling_type}_markov_lengths.csv'

# use mode='a' to add to file instead of overwrite, and 'w' for new doc/overwrite
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write header
    writer.writerow(['Initial Temperature', 'Cooling Rate', 'Markov Length', 'Cooling Type', 'Max Iterations', 'Distances', 'Temperatures'])

    # Run simulations and write to CSV file
    for markov_length in markov_lengths:
        for run in range(1, num_runs + 1):
            best_solution, distances, temperatures, iterations = simulated_annealing(cities, initial_temperature, \
                final_temperature, N, markov_length, cooling_type)

            row_data = [initial_temperature, markov_length, cooling_type, iterations, distances, temperatures]
            writer.writerow(row_data)
    
            
            time.sleep(1)

print(f"Simulation completed. Results saved to {csv_filename}.")